In [1]:
# Load dependencies
import os
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
"""
Desc: Calculate the grouped average
Input: df (pandas DataFrame), 
       gbCols (list of column names to groupby),
       avgCols (list of column names to average),
       sortCols (list of column names to sort by),
       asc (boolean) - ascending or descending sort
Output: x (pandas DataFrame) - (sorted) dataframe with averages of specified columns
"""
def df_gb_avg(df, gbCols, avgCols, sortCols, asc=False):
    
    x = pd.DataFrame(df.groupby(gbCols)[avgCols].mean())
    
    x = x.sort_values(by=sortCols, ascending=asc)
    
    display(x)
    
    return x


"""
Desc: Calculate the grouped sum
Input: df (pandas DataFrame), 
       gbCols (list of column names to groupby),
       avgCols (list of column names to average),
       sortCols (list of column names to sort by),
       asc (boolean) - ascending or descending sort
Output: x (pandas DataFrame) - (sorted) dataframe with averages of specified columns
"""
def df_gb_sum(df, gbCols, avgCols, sortCols, asc=False):
    
    x = pd.DataFrame(df.groupby(gbCols)[avgCols].sum())
    
    x = x.sort_values(by=sortCols, ascending=asc)
    
    display(x)
    
    return x


"""
Desc: Count the number of times a value changes
Input: x (pandas Series)
Output: Total number of times a value changed in the Series
"""
def numChanges(x):
    return sum(x.iloc[:-1] != x.shift(-1).iloc[:-1])

In [3]:
# Load data
incidents = pd.read_csv("INCIDENTS.csv", low_memory=False)
incOwnHist = pd.read_csv("INCIDENT_OWNER_HISTORY.csv", low_memory=False)

# Display
display(incidents.head())

# Print length
print(len(incidents))

# Display
display(incOwnHist.head())

# Print length
print(len(incOwnHist.index))

,TICKET_NMBR,PARENT_SERVICE,service,org_id,ASSIGNED_GROUP,OPEN_DATE,CLOSE_DATE,PRIORITY,STATUS,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS,AGING,CLASS_STRUCTURE_ID,class_structure,CLASSIFICATION_ID,classification,EXTERNAL_SYSTEM,GLOBAL_TICKET_ID,CLOSURE_CODE,LAST_MODIFIED_DATE
0,IN10028612,Middleware,Middleware,1342.0,DC000152,01APR2016:00:01:16,01APR2016:08:26:30,Low,CLOSED,8.42,1.43,NaN,2101,IT Issue \ Hardware,2101,Hardware Issue,EVENTMANAGEMENT,NaN,OTH,12APR2016:05:30:11
1,IN10028613,Midrange,Midrange - Windows -X86,1342.0,NDS00032,01APR2016:00:35:21,11APR2016:11:06:12,Low,CLOSED,250.51,64.10,NaN,21,IT Issue,21,IT Issue,EVENTMANAGEMENT,NaN,CIS,22APR2016:05:30:33
2,IN10028614,Mainframe,Mainframe,1065.0,DC000155,01APR2016:01:43:26,01APR2016:10:28:51,Low,CLOSED,8.76,3.47,NaN,3102,IT Request \ Software Request,3102,Software Request,CREATEDFROMSR,NaN,SUP,12APR2016:05:30:09
3,IN10028615,High-performance Computing,High-performance Computing,1342.0,DC000133,01APR2016:02:57:00,01APR2016:07:42:08,Low,CLOSED,4.75,0.70,NaN,3102,IT Request \ Software Request,3102,Software Request,EVENTMANAGEMENT,NaN,OSD,12APR2016:05:30:10
4,IN10028616,NaN,-1,1342.0,NaN,01APR2016:05:21:02,01APR2016:05:23:39,Low,CLOSED,0.04,0.00,NaN,2103,IT Issue \ Networks,2103,Networks Issue,PHONECALL,NaN,AUT,11APR2016:05:30:07


226296


,ticket_nmbr,STATUS,assigned_group,PARENT_SERVICE,service,CHANGE_DATE,TIME_IN_STATUS_BY_OWNER_HRS
0,IN10028612,QUEUED,DC000152,Middleware,Middleware,01APR2016:00:03:12,8.383333
1,IN10028612,RESOLVED,DC000152,Middleware,Middleware,01APR2016:08:26:30,0.000000
2,IN10028612,CLOSED,DC000152,Middleware,Middleware,12APR2016:05:30:11,NaN
3,IN10028613,QUEUED,NDS00032,Midrange,Midrange - Windows -X86,01APR2016:01:04:45,64.100000
4,IN10028613,RESOLVED,NDS00032,Midrange,Midrange - Windows -X86,11APR2016:11:06:12,0.000000


1152409


In [4]:
# Drop NaNs in incOwnHist
df = incOwnHist[incOwnHist["TIME_IN_STATUS_BY_OWNER_HRS"].notna()]

# Get number of reassignments
ra = pd.DataFrame(df.groupby(["ticket_nmbr"], as_index=False).agg({'STATUS': lambda x: list(dict.fromkeys(x)),
                                                                   'PARENT_SERVICE': lambda x: list(dict.fromkeys(x)),
                                                                   'service': lambda x: list(dict.fromkeys(x)),
                                                                   'TIME_IN_STATUS_BY_OWNER_HRS' : 'sum',
                                                                   'assigned_group' : numChanges
                                                                 }))

# Rename columns
ra.columns = ["TICKET_NMBR", "STATUSES", "PARENT_SERVICES", "services","TIME_IN_STATUS_BY_OWNER_HRS", "reassign_num"]

# Get number of reassignments
df2 = pd.DataFrame(df.groupby(["ticket_nmbr"], as_index=False).agg({'assigned_group' : lambda x: list(dict.fromkeys(x))}))

# Rename columns
df2.columns = ["TICKET_NMBR", "assigned_groups"]

# Merge ra with df2
df = ra.merge(df2)

# Display
display(df.head())

,TICKET_NMBR,STATUSES,PARENT_SERVICES,services,TIME_IN_STATUS_BY_OWNER_HRS,reassign_num,assigned_groups
0,IN10028612,"[QUEUED, RESOLVED]",[Middleware],[Middleware],8.383333,0,[DC000152]
1,IN10028613,"[QUEUED, RESOLVED]",[Midrange],[Midrange - Windows -X86],64.100000,0,[NDS00032]
2,IN10028614,"[QUEUED, INPROG, RESOLVED]",[Mainframe],[Mainframe],8.750000,0,[DC000155]
3,IN10028615,"[QUEUED, INPROG, RESOLVED]",[High-performance Computing],[High-performance Computing],3.616667,0,[DC000133]
4,IN10028617,"[QUEUED, RESOLVED]",[Midrange],[Midrange - Windows -X86],116.150000,0,[DC000115]


In [5]:
# Add columns to count number of status, parent_services, service, and assigned groups
df["num_status"] = df["STATUSES"].str.len()
df["num_PS"] = df["PARENT_SERVICES"].str.len()
df["num_service"] = df["services"].str.len()
df["num_AG"] = df["assigned_groups"].str.len()

# Merge df with incidents
df = incidents.merge(df)

# Display
display(df)

,TICKET_NMBR,PARENT_SERVICE,service,org_id,ASSIGNED_GROUP,OPEN_DATE,CLOSE_DATE,PRIORITY,STATUS,ACTUAL_COMPLETION_HRS,...,STATUSES,PARENT_SERVICES,services,TIME_IN_STATUS_BY_OWNER_HRS,reassign_num,assigned_groups,num_status,num_PS,num_service,num_AG
0,IN10028612,Middleware,Middleware,1342.0,DC000152,01APR2016:00:01:16,01APR2016:08:26:30,Low,CLOSED,8.42,...,"[QUEUED, RESOLVED]",[Middleware],[Middleware],8.383333,0,[DC000152],2,1,1,1
1,IN10028613,Midrange,Midrange - Windows -X86,1342.0,NDS00032,01APR2016:00:35:21,11APR2016:11:06:12,Low,CLOSED,250.51,...,"[QUEUED, RESOLVED]",[Midrange],[Midrange - Windows -X86],64.100000,0,[NDS00032],2,1,1,1
2,IN10028614,Mainframe,Mainframe,1065.0,DC000155,01APR2016:01:43:26,01APR2016:10:28:51,Low,CLOSED,8.76,...,"[QUEUED, INPROG, RESOLVED]",[Mainframe],[Mainframe],8.750000,0,[DC000155],3,1,1,1
3,IN10028615,High-performance Computing,High-performance Computing,1342.0,DC000133,01APR2016:02:57:00,01APR2016:07:42:08,Low,CLOSED,4.75,...,"[QUEUED, INPROG, RESOLVED]",[High-performance Computing],[High-performance Computing],3.616667,0,[DC000133],3,1,1,1
4,IN10028617,Midrange,Midrange - Windows -X86,1342.0,DC000115,01APR2016:06:21:10,18APR2016:13:09:13,Medium,CLOSED,414.80,...,"[QUEUED, RESOLVED]",[Midrange],[Midrange - Windows -X86],116.150000,0,[DC000115],2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224413,IN10293549,Directory Services,Directory Services,1213.0,ITS00379,30NOV2021:23:19:38,01DEC2021:07:51:00,Low,CLOSED,8.52,...,"[QUEUED, RESOLVED]","[Other Activities, Directory Services]","[Other - Service Management, Directory Services]",0.000000,1,"[ESI00011, ITS00379]",2,2,2,2
224414,IN10293550,Storage,Storage - File,1042.0,DC000263,30NOV2021:23:19:43,01DEC2021:12:09:08,Low,CLOSED,12.82,...,"[INPROG, RESOLVED]",[Storage],[Storage - File],0.000000,0,[DC000263],2,1,1,1
224415,IN10293551,Middleware,Middleware,1047.0,DC000152,30NOV2021:23:39:31,01DEC2021:05:46:49,Low,CLOSED,6.12,...,[RESOLVED],[Middleware],[Middleware],0.000000,0,[DC000152],1,1,1,1
224416,IN10293552,Mainframe,Mainframe,1047.0,DC000155,30NOV2021:23:38:56,01DEC2021:03:10:13,Medium,CLOSED,3.52,...,"[INPROG, RESOLVED]","[Other Activities, Mainframe]","[Other - Service Management, Mainframe]",0.000000,1,"[SM000562, DC000155]",2,2,2,2


In [6]:
# Save to csv
df.to_csv("merged_ssc_ticket_data.csv")

In [7]:
# Create folders if they do not already exist
if not os.path.exists("./AVGDATA/"):
    os.mkdir("./AVGDATA/")

if not os.path.exists("./AVGDATA_JEM/"):
    os.mkdir("./AVGDATA_JEM/")
    
if not os.path.exists("./AVGDATA_EEM/"):
    os.mkdir("./AVGDATA_EEM/")

if not os.path.exists("./RA_DATA/"):
    os.mkdir("./RA_DATA/")

In [8]:
# Print how many tickets been reassigned
print("Total number of tickets that have reassigned at least once:", len(df[df["reassign_num"] > 0].index))
print("Percentage of tickets that have been reassigned at least once:", 
      len(df[df["reassign_num"] > 0].index)/len(df.index))
print("Total number of tickets that have not been reassigned:", len(df[df["reassign_num"] == 0].index))
print("Percentage of tickets that have been reassigned at least once:", 
      1 - len(df[df["reassign_num"] > 0].index)/len(df.index))

# Group by different conditions
"""Average number of reassignment"""
df_stat = df_gb_avg(df, ["STATUS"], ["reassign_num"], ["reassign_num"])
df_pr = df_gb_avg(df, ["PRIORITY"], ["reassign_num"], ["reassign_num"])
df_PS = df_gb_avg(df, ["PARENT_SERVICE"], ["reassign_num"], ["reassign_num"])
df_serv = df_gb_avg(df, ["service"], ["reassign_num"], ["reassign_num"])
df_oid = df_gb_avg(df, ["org_id"], ["reassign_num"], ["reassign_num"])
df_AG = df_gb_avg(df, ["ASSIGNED_GROUP"], ["reassign_num"], ["reassign_num"])

"""Avg number of reassignment by Priority"""
df_pr_stat = df_gb_avg(df, ["PRIORITY", "STATUS"], ["reassign_num"], ["reassign_num"])
df_pr_PS = df_gb_avg(df, ["PRIORITY", "PARENT_SERVICE"], ["reassign_num"], ["reassign_num"])
df_pr_serv = df_gb_avg(df, ["PRIORITY", "service"], ["reassign_num"], ["reassign_num"])
df_pr_oid = df_gb_avg(df, ["PRIORITY", "org_id"], ["reassign_num"], ["reassign_num"])
df_pr_AG = df_gb_avg(df, ["PRIORITY", "ASSIGNED_GROUP"], ["reassign_num"], ["reassign_num"])

# Save to csv
df_stat.to_csv("./RA_DATA/df_stat_avg.csv")
df_pr.to_csv("./RA_DATA/df_pr_avg.csv")
df_PS.to_csv("./RA_DATA/df_PS_avg.csv")
df_serv.to_csv("./RA_DATA/df_serv_avg.csv")
df_oid.to_csv("./RA_DATA/df_oid_avg.csv")
df_AG.to_csv("./RA_DATA/df_AG_avg.csv")

df_pr_stat.to_csv("./RA_DATA/df_stat_avg.csv")
df_pr_stat.to_csv("./RA_DATA/df_stat_avg.csv")
df_pr_stat.to_csv("./RA_DATA/df_stat_avg.csv")
df_pr_stat.to_csv("./RA_DATA/df_stat_avg.csv")
df_pr_stat.to_csv("./RA_DATA/df_stat_avg.csv")

Total number of tickets that have reassigned at least once: 58094
Percentage of tickets that have been reassigned at least once: 0.2588651534190662
Total number of tickets that have not been reassigned: 166324
Percentage of tickets that have been reassigned at least once: 0.7411348465809338


,reassign_num
STATUS,
AWAITCHG,1.400000
AWAITCUS,1.000000
SLAHOLD,0.942857
QUEUED,0.844828
INPROG,0.590909
CLOSED,0.435960
RESOLVED,0.410256
AWAITVEN,0.285714


,reassign_num
PRIORITY,
High,0.791935
Medium,0.542302
Low,0.385898


,reassign_num
PARENT_SERVICE,
IT Continuity Support Service,3.000000
WTD Provisioning,2.166667
Solutions Integration Service (SIS),2.000000
Internal Credential Management (ICM),1.271959
Satellite,1.190476
Internet,1.000000
Other Activities,0.855451
Directory Services,0.783923
Cloud Brokering,0.775194


,reassign_num
service,
IT Continuity Support Service,3.000000
WTD - Hardware,2.166667
To be determined,2.000000
Solutions Integration Service (SIS),2.000000
Fixed - Calling Card,2.000000
...,...
High-performance Computing,0.141088
HPC -Big Data Exchange,0.102617
Service Management - Enterprise Control Center (ECC),0.000000


...

reassign_num
PRIORITY STATUS                
High     AWAITCUS      5.000000
         INPROG        2.000000
Low      AWAITCHG      1.750000
Medium   SLAHOLD       1.187500
         QUEUED        1.034483
         AWAITCUS      0.913043
         RESOLVED      0.900000
         INPROG        0.846154
High     CLOSED        0.790594
Low      SLAHOLD       0.736842
         AWAITCUS      0.714286
         QUEUED        0.655172
Medium   AWAITVEN      0.545455
         CLOSED        0.541586
Low      CLOSED        0.385827
         INPROG        0.384615
         RESOLVED      0.241379
         AWAITVEN      0.210526
Medium   AWAITCHG      0.000000

reassign_num
PRIORITY PARENT_SERVICE                                   
High     Cloud Brokering                          8.000000
Medium   IT Continuity Support Service            3.000000
High     Mainframe                                2.428571
Low      WTD Provisioning                         2.166667
         Solutions Integration Service (SIS)      2.000000
...                                                    ...
Medium   Facilities-as-a-service (FaaS)           0.000000
         Bulk Print                               0.000000
Low      Bulk Print                               0.000000
High     Distributed Print                        0.000000
         Classified Infrastructure                0.000000

[85 rows x 1 columns]

reassign_num
PRIORITY service                                                         
High     Cloud Brokering                                         8.000000
         -1                                                      3.000000
Medium   IT Continuity Support Service                           3.000000
High     Mainframe                                               2.666667
Low      WTD - Hardware                                          2.166667
...                                                                   ...
         Service Management - Enterprise Control Center ...      0.000000
Medium   Bulk Print                                              0.000000
         Faciltiies-as-a-service (FaaS)                          0.000000
         Fixed - Key Systems and Interchange                     0.000000
High     Classified Infrastructure - GCSI - Hosting Envi...      0.000000

[170 rows x 1 columns]

reassign_num
PRIORITY org_id               
Low      918.0            11.0
High     1068.0            4.5
Medium   1232.0            3.0
         867.0             3.0
High     17498.0           3.0
...                        ...
         1165.0            0.0
         1202.0            0.0
         1732.0            0.0
         1247.0            0.0
         1243.0            0.0

[181 rows x 1 columns]

reassign_num
PRIORITY ASSIGNED_GROUP              
High     ITS00347                10.0
         DC000245                10.0
         DC000121                 8.0
Medium   ITS00350                 8.0
         COR00559                 5.0
...                               ...
         ITS00387                 0.0
Low      ITS00350                 0.0
         ITS00371                 0.0
         ITS00390                 0.0
Medium   TIVMON                   0.0

[990 rows x 1 columns]

In [9]:
"""Sum reassignment number"""
df_stat = df_gb_sum(df, ["STATUS"], ["reassign_num"], ["reassign_num"])
df_pr = df_gb_sum(df, ["PRIORITY"], ["reassign_num"], ["reassign_num"])
df_PS = df_gb_sum(df, ["PARENT_SERVICE"], ["reassign_num"], ["reassign_num"])
df_serv = df_gb_sum(df, ["service"], ["reassign_num"], ["reassign_num"])
df_oid = df_gb_sum(df, ["org_id"], ["reassign_num"], ["reassign_num"])
df_AG = df_gb_sum(df, ["ASSIGNED_GROUP"], ["reassign_num"], ["reassign_num"])

"""Sum reassignment number by Priority"""
df_pr_stat = df_gb_sum(df, ["PRIORITY", "STATUS"], ["reassign_num"], ["reassign_num"])
df_pr_PS = df_gb_sum(df, ["PRIORITY", "PARENT_SERVICE"], ["reassign_num"], ["reassign_num"])
df_pr_serv = df_gb_sum(df, ["PRIORITY", "service"], ["reassign_num"], ["reassign_num"])
df_pr_oid = df_gb_sum(df, ["PRIORITY", "org_id"], ["reassign_num"], ["reassign_num"])
df_pr_AG = df_gb_sum(df, ["PRIORITY", "ASSIGNED_GROUP"], ["reassign_num"], ["reassign_num"])

# Save to csv
df_stat.to_csv("./RA_DATA/df_stat_sum.csv")
df_pr.to_csv("./RA_DATA/df_pr_sum.csv")
df_PS.to_csv("./RA_DATA/df_PS_sum.csv")
df_serv.to_csv("./RA_DATA/df_serv_sum.csv")
df_oid.to_csv("./RA_DATA/df_oid_sum.csv")
df_AG.to_csv("./RA_DATA/df_AG_sum.csv")

df_pr_stat.to_csv("./RA_DATA/df_stat_sum.csv")
df_pr_stat.to_csv("./RA_DATA/df_stat_sum.csv")
df_pr_stat.to_csv("./RA_DATA/df_stat_sum.csv")
df_pr_stat.to_csv("./RA_DATA/df_stat_sum.csv")
df_pr_stat.to_csv("./RA_DATA/df_stat_sum.csv")

,reassign_num
STATUS,
CLOSED,97714
QUEUED,49
INPROG,39
SLAHOLD,33
AWAITCUS,31
RESOLVED,16
AWAITVEN,14
AWAITCHG,7


,reassign_num
PRIORITY,
Low,61210
Medium,33492
High,3201


,reassign_num
PARENT_SERVICE,
Other Activities,25939
Midrange,12043
Intra-building Network Services,11636
GC WAN,10139
Middleware,5617
Mobile Devices,4147
Fixed Line,3796
Directory Services,3345
Firewall,3109


,reassign_num
service,
Other - Service Management,21022
GC WAN,9790
IBN - GC LAN,7398
Midrange - Windows -X86,6742
Middleware,5617
...,...
IBN - Cabling,1
Internet,1
Bulk Print,0


,reassign_num
org_id,
1342.0,25889
1047.0,12756
1028.0,6914
1062.0,6595
1045.0,5079
...,...
837.0,1
1183.0,1
1233.0,1


...

reassign_num
PRIORITY STATUS                
Low      CLOSED           61135
Medium   CLOSED           33385
High     CLOSED            3194
Medium   QUEUED              30
         INPROG              22
         AWAITCUS            21
         SLAHOLD             19
Low      QUEUED              19
         INPROG              15
         SLAHOLD             14
Medium   RESOLVED             9
Low      AWAITVEN             8
         RESOLVED             7
         AWAITCHG             7
Medium   AWAITVEN             6
Low      AWAITCUS             5
High     AWAITCUS             5
         INPROG               2
Medium   AWAITCHG             0

reassign_num
PRIORITY PARENT_SERVICE                               
Low      Other Activities                        17979
         Midrange                                 8031
Medium   Other Activities                         7342
Low      Intra-building Network Services          6440
Medium   GC WAN                                   5180
...                                                ...
High     Distributed Print                           0
Low      Bulk Print                                  0
Medium   Bulk Print                                  0
         Facilities-as-a-service (FaaS)              0
High     Classified Infrastructure                   0

[85 rows x 1 columns]

reassign_num
PRIORITY service                                                         
Low      Other - Service Management                                 14636
Medium   Other - Service Management                                  5946
         GC WAN                                                      4934
Low      GC WAN                                                      4571
         Midrange - Windows -X86                                     4213
...                                                                   ...
High     Distributed Print                                              0
Low      Service Management - Enterprise Control Center ...             0
         Bulk Print                                                     0
Medium   Bulk Print                                                     0
High     GC WAN International                                           0

[170 rows x 1 columns]

reassign_num
PRIORITY org_id              
Low      1342.0         16441
Medium   1342.0          8769
Low      1047.0          7638
         1028.0          5626
         1062.0          4761
...                       ...
Medium   1337.0             0
         988.0              0
High     1247.0             0
         1243.0             0
         1202.0             0

[181 rows x 1 columns]

reassign_num
PRIORITY ASSIGNED_GROUP              
Low      ESI00011               12036
Medium   ESI00011                4619
         NW000417                2408
Low      DC000152                2290
         NW000417                1838
...                               ...
         ESI00046                   0
         EU000204                   0
         ITS00109                   0
         ITS00315                   0
Medium   TIVMON                     0

[990 rows x 1 columns]

# Incidents Owner History Data

In [10]:
# Average time incidents spend in particular status
avgTimeStatus = df_gb_avg(incOwnHist, ['STATUS'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeStatus.to_csv("./AVGDATA/IOH_avgTimeStatus.csv")

# Average time incidents spend with assigned group
avgTimeAG = df_gb_avg(incOwnHist, ['assigned_group'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA/IOH_avgTimeAG.csv")

# Average time incidents spend by parent service
avgTimePS = df_gb_avg(incOwnHist, ['PARENT_SERVICE'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimePS.to_csv("./AVGDATA/IOH_avgTimePS.csv")

# Average time incidents spend by service
avgTimeServ = df_gb_avg(incOwnHist, ['service'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeServ.to_csv("./AVGDATA/IOH_avgTimeServ.csv")

,TIME_IN_STATUS_BY_OWNER_HRS
STATUS,
PENDINGREV,209.543013
PENDINGCHG,158.836006
AWAITVEN,144.054339
SLAHOLD,128.120452
AWAITCHG,127.377065
PENDINGVEN,102.631822
PENDINGCUS,92.318825
AWAITCUS,85.690441
PENDING,68.230397


,TIME_IN_STATUS_BY_OWNER_HRS
assigned_group,
ITS00386,2128.316667
SM000541,1973.583333
EDC00080,1771.750000
NW000468,1766.233333
NSD00012,1639.316667
...,...
DC000265,NaN
ITS00400,NaN
NW000501,NaN


,TIME_IN_STATUS_BY_OWNER_HRS
PARENT_SERVICE,
IT Continuity Support Service,1973.583333
Bulk Print,377.007738
Solutions Integration Service (SIS),303.705000
WTD Provisioning,175.387931
Satellite,135.465563
Intra-building Network Services,69.585276
Data Centre Facilities Management,68.552445
Cloud Brokering,63.118243
Firewall,61.067085


,TIME_IN_STATUS_BY_OWNER_HRS
service,
IT Continuity Support Service,1973.583333
WTD Provisioning,625.816667
Solutions Integration Service (SIS),428.064286
Bulk Print,377.007738
IBN - Cabling,346.957143
...,...
Managed Secure File Transfer,13.037097
Mobile - Voice & Data,12.936081
Mainframe,12.075492


In [11]:
"""Group By Status"""
# Average time incidents spend with assigned group
avgTimeAG = df_gb_avg(incOwnHist, ['assigned_group', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA/IOH_avgTimeAGStatus.csv")

# Average time incidents spend by parent service
avgTimePS = df_gb_avg(incOwnHist, ['PARENT_SERVICE', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimePS.to_csv("./AVGDATA/IOH_avgTimePSStatus.csv")

# Average time incidents spend by service
avgTimeServ = df_gb_avg(incOwnHist, ['service', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeServ.to_csv("./AVGDATA/IOH_avgTimeServStatus.csv")

,,TIME_IN_STATUS_BY_OWNER_HRS
assigned_group,STATUS,
ESI00052,AWAITCUS,5550.833333
SM000541,INPROG,3855.800000
NW000468,INPROG,3530.733333
ITS00302,INPROG,3306.866667
SM000575,INPROG,2972.000000
...,...,...
SM000582,CLOSED,NaN
SM000583,QUEUED,NaN
SM000585,CLOSED,NaN


,,TIME_IN_STATUS_BY_OWNER_HRS
PARENT_SERVICE,STATUS,
IT Continuity Support Service,INPROG,3855.800000
Middleware,PENDINGCHG,1504.883333
Database,PENDINGCHG,1503.183333
Mobile Devices,PENDINGCHG,1173.000000
Toll-free Voice,PENDING,931.566667
...,...,...
Solutions Integration Service (SIS),CLOSED,NaN
Storage,CLOSED,NaN
Toll-free Voice,CLOSED,NaN


,,TIME_IN_STATUS_BY_OWNER_HRS
service,STATUS,
IT Continuity Support Service,INPROG,3855.800000
Other - Project Management & Delivery,AWAITCHG,1743.341667
Middleware,PENDINGCHG,1504.883333
Database,PENDINGCHG,1503.183333
Solutions Integration Service (SIS),INPROG,1485.216667
...,...,...
Storage - File,CLOSED,NaN
To be determined,CLOSED,NaN
Toll-free Voice,CLOSED,NaN


# Incident Data

In [12]:
# Average completion time by status
avgCmplStatus = df_gb_avg(incidents, ['STATUS'],
                          ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                          ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplStatus.to_csv("./AVGDATA/Inc_avgCmplStatus.csv")

# Average completion time by parent service
avgCmplPS = df_gb_avg(incidents, ['PARENT_SERVICE'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA/Inc_avgCmplPS.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(incidents, ['service'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA/Inc_avgCmplServ.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(incidents, ['ASSIGNED_GROUP'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA/Inc_avgCmplAG.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(incidents, ['org_id'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(incidents, ['org_id'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA/Inc_avgCmplORG.csv")
avgBizORG.to_csv("./AVGDATA/Inc_avgBizORG.csv")

,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
STATUS,,
RESOLVED,1926.444107,553.146786
INPROG,1791.134000,508.100000
CLOSED,263.471013,80.581383
SLAHOLD,98.186667,28.856667
QUEUED,16.035000,9.025000
AWAITVEN,1.590833,0.238333
AWAITCHG,NaN,NaN
AWAITCUS,NaN,NaN


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
PARENT_SERVICE,,
IT Continuity Support Service,14735.130000,4219.110000
Solutions Integration Service (SIS),4071.950000,1172.520000
Satellite,1750.180732,501.652927
Bulk Print,947.716667,269.763333
Firewall,490.807570,141.575320
Toll-free Voice,482.581790,136.183231
Contact Centre,454.087896,133.452783
Intra-building Network Services,450.760953,129.122458
Conferencing Services,447.047643,126.918488


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
service,,
IT Continuity Support Service,14735.130000,4219.110000
Solutions Integration Service (SIS),4071.950000,1172.520000
Intra-building Network Services,2167.273525,620.715362
Satellite,1750.180732,501.652927
Mainframe - zOS & zLinux,1683.035082,480.606475
...,...,...
Other - Telecom & Network,78.888257,21.543248
Middleware,59.232249,15.976003
Internet,57.955714,16.114286


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
ASSIGNED_GROUP,,
DC000157,16104.71,4610.70
NW000435,15457.16,4431.16
SM000541,14735.13,4219.11
DC000213,10588.50,3034.49
SM000575,10558.17,3020.00
...,...,...
NW000516,NaN,NaN
PM000620,NaN,NaN
SM000552,NaN,NaN


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,,
918.0,2984.790,843.215000
1237.0,2567.195,732.470000
948.0,1482.200,416.270000
1083.0,1417.575,413.983750
1085.0,1388.190,393.813333
...,...,...
988.0,22.405,5.785000
1732.0,14.980,8.200000
1232.0,10.060,0.000000


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,,
918.0,2984.790,843.215000
1237.0,2567.195,732.470000
948.0,1482.200,416.270000
1083.0,1417.575,413.983750
1085.0,1388.190,393.813333
...,...,...
1732.0,14.980,8.200000
988.0,22.405,5.785000
837.0,0.780,0.770000


In [13]:
"""Group by Priority"""
# Average completion time by status
avgCmplStatus = df_gb_avg(incidents, ['STATUS', 'PRIORITY'],
                          ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                          ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplStatus.to_csv("./AVGDATA/Inc_avgCmplStatus_pr.csv")

# Average completion time by parent service
avgCmplPS = df_gb_avg(incidents, ['PARENT_SERVICE', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA/Inc_avgCmplPS_pr.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(incidents, ['service', 'PRIORITY'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA/Inc_avgCmplServ_pr.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(incidents, ['ASSIGNED_GROUP', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA/Inc_avgCmplAG_pr.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(incidents, ['org_id', 'PRIORITY'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(incidents, ['org_id', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA/Inc_avgCmplORG_pr.csv")
avgBizORG.to_csv("./AVGDATA/Inc_avgBizORG_pr.csv")

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
STATUS   PRIORITY                                                
RESOLVED Low                 2189.252234               627.426383
INPROG   Medium              1905.213333               547.850000
         Low                 1620.015000               448.475000
RESOLVED Medium              1592.606757               458.791622
CLOSED   High                 390.950069               390.950069
         Low                  265.415804                75.782776
         Medium               250.074190                72.515202
SLAHOLD  Medium               145.675000                41.675000
QUEUED   Low                   28.370000                14.370000
AWAITVEN Medium                 8.396667                 0.000000
QUEUED   Medium                 3.700000                 3.680000
SLAHOLD  Low                    3.210000                 3.220000
AWAITVEN Low                   -0.677778                 0.317778
AWAITCHG Low                         NaN                      NaN
         Medium                      NaN                      NaN
AWAITCUS High                        NaN                      NaN
         Low                         NaN                      NaN
         Medium                      NaN                      NaN
INPROG   High                        NaN                      NaN
QUEUED   High                        NaN                      NaN
SLAHOLD  High                        NaN                      NaN

ACTUAL_COMPLETION_HRS  \
PARENT_SERVICE                      PRIORITY                          
IT Continuity Support Service       Medium             14735.130000   
Solutions Integration Service (SIS) Low                 4071.950000   
Other Activities                    High                2889.018672   
Satellite                           Medium              1934.070000   
                                    Low                 1674.088621   
...                                                             ...   
Database                            High                   6.059524   
Internet                            Medium                 3.485000   
Facilities-as-a-service (FaaS)      Medium                 3.460000   
Bulk Print                          Medium                 1.395000   
Distributed Print                   High                   0.410000   

                                              BUSINESS_COMPLETION_HRS  
PARENT_SERVICE                      PRIORITY                           
IT Continuity Support Service       Medium                4219.110000  
Solutions Integration Service (SIS) Low                   1172.520000  
Other Activities                    High                  2889.018672  
Satellite                           Medium                 556.036667  
                                    Low                    479.149310  
...                                                               ...  
Database                            High                     6.059524  
Internet                            Medium                   3.490000  
Facilities-as-a-service (FaaS)      Medium                   3.470000  
Bulk Print                          Medium                   1.400000  
Distributed Print                   High                     0.410000  

[86 rows x 2 columns]

,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
service,PRIORITY,,
IT Continuity Support Service,Medium,14735.130000,4219.110000
Other - Service Management,High,4182.539430,4182.539430
Solutions Integration Service (SIS),Low,4071.950000,1172.520000
Mainframe - zOS & zLinux,Medium,2985.219048,853.122857
Intra-building Network Services,Low,2541.930014,726.203804
...,...,...,...
Faciltiies-as-a-service (FaaS),Medium,3.460000,3.470000
Bulk Print,Medium,1.395000,1.400000
Distributed Print,High,0.410000,0.410000


,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
ASSIGNED_GROUP,PRIORITY,,
EDC00031,High,255749.272,255749.272
DC000157,Medium,16104.710,4610.700
NW000435,Low,15457.160,4431.160
SM000541,Medium,14735.130,4219.110
NW000441,Medium,13967.890,4019.900
...,...,...,...
PM000620,Low,NaN,NaN
SM000500,Medium,NaN,NaN
SM000552,Low,NaN,NaN


,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,PRIORITY,,
918.0,Low,5966.890000,1683.750000
1237.0,Low,4763.150000,1364.160000
1083.0,Low,1874.788333,545.903333
1085.0,Medium,1648.095000,471.575000
1260.0,Medium,1486.845000,423.850000
...,...,...,...
918.0,Medium,2.690000,2.680000
1202.0,High,2.210000,2.210000
837.0,Medium,0.780000,0.770000


,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,PRIORITY,,
918.0,Low,5966.890000,1683.750000
1032.0,High,1409.740000,1409.740000
1237.0,Low,4763.150000,1364.160000
1342.0,High,1173.525417,1173.525417
1303.0,High,615.028571,615.028571
...,...,...,...
1732.0,Low,15.310000,1.750000
837.0,Medium,0.780000,0.770000
988.0,Low,0.260000,0.250000


In [14]:
"""Group by Priority and Status"""
# Average completion time by parent service
avgCmplPS = df_gb_avg(incidents, ['PARENT_SERVICE', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA/Inc_avgCmplPS_pr_stat.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(incidents, ['service', 'PRIORITY', 'STATUS'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA/Inc_avgCmplServ_pr_stat.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(incidents, ['ASSIGNED_GROUP', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA/Inc_avgCmplAG_pr_stat.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(incidents, ['org_id', 'PRIORITY', 'STATUS'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(incidents, ['org_id', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA/Inc_avgCmplORG_pr_stat.csv")
avgBizORG.to_csv("./AVGDATA/Inc_avgBizORG_pr_stat.csv")

ACTUAL_COMPLETION_HRS  \
PARENT_SERVICE                       PRIORITY STATUS                            
IT Continuity Support Service        Medium   CLOSED             14735.130000   
Midrange                             Medium   RESOLVED            6480.886667   
Internal Credential Management (ICM) Medium   INPROG              5571.140000   
Solutions Integration Service (SIS)  Low      CLOSED              4071.950000   
GC WAN                               Low      RESOLVED            3144.222000   
...                                                                       ...   
Storage                              Medium   INPROG                      NaN   
                                              QUEUED                      NaN   
                                              SLAHOLD                     NaN   
Toll-free Voice                      Medium   AWAITCUS                    NaN   
                                              QUEUED                      NaN   

                                                        BUSINESS_COMPLETION_HRS  
PARENT_SERVICE                       PRIORITY STATUS                             
IT Continuity Support Service        Medium   CLOSED                4219.110000  
Midrange                             Medium   RESOLVED              1860.883333  
Internal Credential Management (ICM) Medium   INPROG                1603.150000  
Solutions Integration Service (SIS)  Low      CLOSED                1172.520000  
GC WAN                               Low      RESOLVED               903.798000  
...                                                                         ...  
Storage                              Medium   INPROG                        NaN  
                                              QUEUED                        NaN  
                                              SLAHOLD                       NaN  
Toll-free Voice                      Medium   AWAITCUS                      NaN  
                                              QUEUED                        NaN  

[268 rows x 2 columns]

ACTUAL_COMPLETION_HRS  \
service                              PRIORITY STATUS                            
IT Continuity Support Service        Medium   CLOSED             14735.130000   
Midrange - Windows -X86              Medium   RESOLVED            6480.886667   
Data Centre Facilities Management    Medium   RESOLVED            6382.660000   
Internal Credential Management (ICM) Medium   INPROG              5571.140000   
Other - Internal Services & Strategy Low      RESOLVED            4613.043333   
...                                                                       ...   
Storage - File                       Low      INPROG                      NaN   
                                     Medium   AWAITCUS                    NaN   
                                              INPROG                      NaN   
Toll-free Voice                      Medium   AWAITCUS                    NaN   
                                              QUEUED                      NaN   

                                                        BUSINESS_COMPLETION_HRS  
service                              PRIORITY STATUS                             
IT Continuity Support Service        Medium   CLOSED                4219.110000  
Midrange - Windows -X86              Medium   RESOLVED              1860.883333  
Data Centre Facilities Management    Medium   RESOLVED              1818.670000  
Internal Credential Management (ICM) Medium   INPROG                1603.150000  
Other - Internal Services & Strategy Low      RESOLVED              1321.710000  
...                                                                         ...  
Storage - File                       Low      INPROG                        NaN  
                                     Medium   AWAITCUS                      NaN  
                                              INPROG                        NaN  
Toll-free Voice                      Medium   AWAITCUS                      NaN  
                                              QUEUED                        NaN  

[444 rows x 2 columns]

ACTUAL_COMPLETION_HRS  \
ASSIGNED_GROUP PRIORITY STATUS                            
EDC00031       High     CLOSED               255749.272   
DC000157       Medium   RESOLVED              16104.710   
NW000435       Low      CLOSED                15457.160   
SM000541       Medium   CLOSED                14735.130   
NW000441       Medium   CLOSED                13967.890   
...                                                 ...   
SM000585       Medium   INPROG                      NaN   
SM000586       Low      AWAITCUS                    NaN   
                        SLAHOLD                     NaN   
               Medium   AWAITCUS                    NaN   
                        SLAHOLD                     NaN   

                                  BUSINESS_COMPLETION_HRS  
ASSIGNED_GROUP PRIORITY STATUS                             
EDC00031       High     CLOSED                 255749.272  
DC000157       Medium   RESOLVED                 4610.700  
NW000435       Low      CLOSED                   4431.160  
SM000541       Medium   CLOSED                   4219.110  
NW000441       Medium   CLOSED                   4019.900  
...                                                   ...  
SM000585       Medium   INPROG                        NaN  
SM000586       Low      AWAITCUS                      NaN  
                        SLAHOLD                       NaN  
               Medium   AWAITCUS                      NaN  
                        SLAHOLD                       NaN  

[1474 rows x 2 columns]

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
org_id PRIORITY STATUS                                                  
1055.0 Medium   RESOLVED            6437.906667              1847.866667
1273.0 Medium   RESOLVED            6382.660000              1818.670000
918.0  Low      CLOSED              5966.890000              1683.750000
1045.0 Medium   RESOLVED            5835.846667              1674.343333
1047.0 Low      RESOLVED            5672.427500              1623.267500
...                                         ...                      ...
1355.0 Medium   AWAITCUS                    NaN                      NaN
1377.0 High     QUEUED                      NaN                      NaN
       Low      INPROG                      NaN                      NaN
       Medium   QUEUED                      NaN                      NaN
1753.0 Medium   QUEUED                      NaN                      NaN

[473 rows x 2 columns]

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
org_id PRIORITY STATUS                                                  
1055.0 Medium   RESOLVED            6437.906667              1847.866667
1273.0 Medium   RESOLVED            6382.660000              1818.670000
918.0  Low      CLOSED              5966.890000              1683.750000
1045.0 Medium   RESOLVED            5835.846667              1674.343333
1047.0 Low      RESOLVED            5672.427500              1623.267500
...                                         ...                      ...
1355.0 Medium   AWAITCUS                    NaN                      NaN
1377.0 High     QUEUED                      NaN                      NaN
       Low      INPROG                      NaN                      NaN
       Medium   QUEUED                      NaN                      NaN
1753.0 Medium   QUEUED                      NaN                      NaN

[473 rows x 2 columns]

# Event Management

In [15]:
"""Just Event Management"""
# Filter for event management tickets
jemInc = incidents[incidents["EXTERNAL_SYSTEM"] == "EVENTMANAGEMENT"]

# Reset index
jemInc = jemInc.reset_index(drop=True)

# Filter Incident owner history for event management tickets
jemIOH = incOwnHist[incOwnHist["ticket_nmbr"].isin(jemInc["TICKET_NMBR"])]

# Reset index
jemIOH = jemIOH.reset_index(drop=True)

# Display
display(jemInc)
display(jemIOH)

,TICKET_NMBR,PARENT_SERVICE,service,org_id,ASSIGNED_GROUP,OPEN_DATE,CLOSE_DATE,PRIORITY,STATUS,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS,AGING,CLASS_STRUCTURE_ID,class_structure,CLASSIFICATION_ID,classification,EXTERNAL_SYSTEM,GLOBAL_TICKET_ID,CLOSURE_CODE,LAST_MODIFIED_DATE
0,IN10028612,Middleware,Middleware,1342.0,DC000152,01APR2016:00:01:16,01APR2016:08:26:30,Low,CLOSED,8.42,1.43,NaN,2101,IT Issue \ Hardware,2101,Hardware Issue,EVENTMANAGEMENT,NaN,OTH,12APR2016:05:30:11
1,IN10028613,Midrange,Midrange - Windows -X86,1342.0,NDS00032,01APR2016:00:35:21,11APR2016:11:06:12,Low,CLOSED,250.51,64.10,NaN,21,IT Issue,21,IT Issue,EVENTMANAGEMENT,NaN,CIS,22APR2016:05:30:33
2,IN10028615,High-performance Computing,High-performance Computing,1342.0,DC000133,01APR2016:02:57:00,01APR2016:07:42:08,Low,CLOSED,4.75,0.70,NaN,3102,IT Request \ Software Request,3102,Software Request,EVENTMANAGEMENT,NaN,OSD,12APR2016:05:30:10
3,IN10028617,Midrange,Midrange - Windows -X86,1342.0,DC000115,01APR2016:06:21:10,18APR2016:13:09:13,Medium,CLOSED,414.80,116.15,NaN,2101,IT Issue \ Hardware,2101,Hardware Issue,EVENTMANAGEMENT,NaN,BSR,29APR2016:05:30:36
4,IN10028618,Other Activities,Other - Service Management,1342.0,ESI00011,01APR2016:06:30:34,17APR2016:19:40:25,Medium,CLOSED,397.16,110.00,NaN,2102,IT Issue \ Software,2102,Software Issue,EVENTMANAGEMENT,NaN,INF,28APR2016:05:30:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96960,IN10293537,Intra-building Network Services,IBN - GC LAN,1047.0,NW000405,30NOV2021:20:54:25,10DEC2021:08:38:54,Medium,RESOLVED,227.74,71.63,NaN,2103,IT Issue \ Networks,2103,Networks Issue,EVENTMANAGEMENT,NaN,RWOI,10DEC2021:08:38:54
96961,IN10293538,Middleware,Middleware,1047.0,DC000152,30NOV2021:21:21:23,03DEC2021:06:42:48,Low,CLOSED,57.36,20.00,NaN,2102,IT Issue \ Software,2102,Software Issue,EVENTMANAGEMENT,NaN,RP,14DEC2021:05:30:16
96962,IN10293540,High-performance Computing,High-performance Computing,1062.0,DC000132,30NOV2021:21:42:14,NaN,Low,AWAITVEN,NaN,NaN,19.658366,2102,IT Issue \ Software,2102,Software Issue,EVENTMANAGEMENT,NaN,RP,30NOV2021:21:51:26
96963,IN10293541,Midrange,Midrange,1342.0,DC000209,30NOV2021:21:44:01,01DEC2021:08:37:06,Low,CLOSED,10.88,1.62,NaN,21,IT Issue,21,IT Issue,EVENTMANAGEMENT,NaN,RP,12DEC2021:05:30:09


,ticket_nmbr,STATUS,assigned_group,PARENT_SERVICE,service,CHANGE_DATE,TIME_IN_STATUS_BY_OWNER_HRS
0,IN10028612,QUEUED,DC000152,Middleware,Middleware,01APR2016:00:03:12,8.383333
1,IN10028612,RESOLVED,DC000152,Middleware,Middleware,01APR2016:08:26:30,0.000000
2,IN10028612,CLOSED,DC000152,Middleware,Middleware,12APR2016:05:30:11,NaN
3,IN10028613,QUEUED,NDS00032,Midrange,Midrange - Windows -X86,01APR2016:01:04:45,64.100000
4,IN10028613,RESOLVED,NDS00032,Midrange,Midrange - Windows -X86,11APR2016:11:06:12,0.000000
...,...,...,...,...,...,...,...
420311,IN10293541,NEW,NaN,NaN,NaN,NaN,NaN
420312,IN10293552,INPROG,SM000562,Other Activities,Other - Service Management,NaN,0.000000
420313,IN10293552,QUEUED,SM000562,Other Activities,Other - Service Management,NaN,NaN
420314,IN10293552,INPROG,DC000155,Mainframe,Mainframe,NaN,NaN


In [16]:
# Average time incidents spend in particular status
avgTimeStatus = df_gb_avg(jemIOH, ['STATUS'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeStatus.to_csv("./AVGDATA_JEM/jemIOH_avgTimeStatus.csv")

# Average time incidents spend with assigned group
avgTimeAG = df_gb_avg(jemIOH, ['assigned_group'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA_JEM/jemIOH_avgTimeAG.csv")

# Average time incidents spend by parent service
avgTimePS = df_gb_avg(jemIOH, ['PARENT_SERVICE'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimePS.to_csv("./AVGDATA_JEM/jemIOH_avgTimePS.csv")

# Average time incidents spend by service
avgTimeAG = df_gb_avg(jemIOH, ['service'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA_JEM/jemIOH_avgTimeServ.csv")

,TIME_IN_STATUS_BY_OWNER_HRS
STATUS,
AWAITVEN,167.117355
PENDINGVEN,88.863389
SLAHOLD,73.841899
AWAITCHG,66.540715
AWAITCUS,58.563833
PENDINGREV,48.973810
PENDINGCHG,40.213333
PENDING,32.912584
PENDINGCUS,23.423246


,TIME_IN_STATUS_BY_OWNER_HRS
assigned_group,
ESI00052,2772.372917
ITS00352,2502.758333
DC000145,1724.883333
NW000469,1639.416667
ITS00329,1422.923016
...,...
EDC00041,0.000000
NW000467,0.000000
DC000226,NaN


,TIME_IN_STATUS_BY_OWNER_HRS
PARENT_SERVICE,
Bulk Print,1724.883333
Distributed Print,319.450000
Classified Infrastructure,290.000000
Firewall,44.564273
Managed Secure File Transfer,26.101613
High-performance Computing,25.617979
Email,24.196342
Database,21.292119
Internal Credential Management (ICM),18.915646


,TIME_IN_STATUS_BY_OWNER_HRS
service,
Bulk Print,1724.883333
Mobile Devices,638.362500
Distributed Print,319.450000
Classified Infrastructure - GCSI - Hosting Environment,290.000000
Mainframe - zOS & zLinux,251.863226
Intra-building Network Services,122.159201
Other - Security,93.827952
Firewall,44.564273
Other - Internal Services & Strategy,42.347312


In [17]:
"""Group By Status"""
# Average time incidents spend with assigned group
avgTimeAG = df_gb_avg(jemIOH, ['assigned_group', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA_JEM/jemIOH_avgTimeAGStatus.csv")

# Average time incidents spend by parent service
avgTimePS = df_gb_avg(jemIOH, ['PARENT_SERVICE', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimePS.to_csv("./AVGDATA_JEM/jemIOH_avgTimePSStatus.csv")

# Average time incidents spend by service
avgTimeAG = df_gb_avg(jemIOH, ['service', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA_JEM/jemIOH_avgTimeServStatus.csv")

TIME_IN_STATUS_BY_OWNER_HRS
assigned_group STATUS                               
DC000118       QUEUED                    5559.466667
ITS00340       AWAITCUS                  5552.816667
ESI00052       AWAITCUS                  5550.833333
ITS00329       AWAITCUS                  5446.583333
               QUEUED                    3490.685714
...                                              ...
SM000562       CLOSED                            NaN
SM000585       AWAITCUS                          NaN
               CLOSED                            NaN
SM000586       CLOSED                            NaN
TIVMON         CLOSED                            NaN

[1480 rows x 1 columns]

TIME_IN_STATUS_BY_OWNER_HRS
PARENT_SERVICE                    STATUS                                 
Bulk Print                        QUEUED                      2111.008333
                                  INPROG                       952.633333
Secure Remote Access              AWAITCUS                     727.808333
Data Centre Facilities Management PENDINGVEN                   584.444444
Email                             AWAITCUS                     570.076667
...                                                                   ...
Other Activities                  CLOSED                              NaN
Secure Remote Access              AWAITCHG                            NaN
                                  CLOSED                              NaN
Storage                           CLOSED                              NaN
Workplace Technology Services     CLOSED                              NaN

[221 rows x 1 columns]

TIME_IN_STATUS_BY_OWNER_HRS
service                       STATUS                               
Bulk Print                    QUEUED                    2111.008333
Other - Security              AWAITCUS                  1468.991667
Mobile Devices                INPROG                    1321.333333
                              QUEUED                    1010.000000
Bulk Print                    INPROG                     952.633333
...                                                             ...
Secure Remote Access          AWAITCHG                          NaN
                              CLOSED                            NaN
Storage                       CLOSED                            NaN
Storage - File                CLOSED                            NaN
Workplace Technology Services CLOSED                            NaN

[365 rows x 1 columns]

In [18]:
# Average completion time by status
avgCmplStatus = df_gb_avg(jemInc, ['STATUS'],
                          ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                          ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplStatus.to_csv("./AVGDATA_JEM/jemInc_avgCmplStatus.csv")

# Average completion time by parent service
avgCmplPS = df_gb_avg(jemInc, ['PARENT_SERVICE'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA_JEM/jemInc_avgCmplPS.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(jemInc, ['service'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA_JEM/jemInc_avgCmplServ.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(jemInc, ['ASSIGNED_GROUP'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA_JEM/jemInc_avgCmplAG.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(jemInc, ['org_id'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(jemInc, ['org_id'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA_JEM/jemInc_avgCmplORG.csv")
avgBizORG.to_csv("./AVGDATA_JEM/jemInc_avgBizORG.csv")

,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
STATUS,,
RESOLVED,2568.350213,737.081915
INPROG,955.990000,264.800000
CLOSED,122.551871,43.918936
QUEUED,28.370000,14.370000
AWAITVEN,4.216667,0.292222
AWAITCUS,NaN,NaN
SLAHOLD,NaN,NaN


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
PARENT_SERVICE,,
Firewall,334.168306,94.297132
Other Activities,311.433446,187.417807
High-performance Computing,228.265096,66.235825
Email,197.513241,55.943657
Internal Credential Management (ICM),173.175082,46.918033
Intra-building Network Services,134.191680,36.432287
Database,113.763271,32.698271
Workplace Technology Services,108.331128,27.255668
Mainframe,101.080406,27.666743


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
service,,
Mainframe - zOS & zLinux,1969.112875,561.594500
Intra-building Network Services,1303.024045,372.335909
Other - Security,494.226139,140.868399
Firewall,334.168306,94.297132
Other - Service Management,320.901298,206.286496
High-performance Computing,314.712877,90.003782
Email,197.513241,55.943657
Internal Credential Management (ICM),173.175082,46.918033
Other - Project Management & Delivery,153.010000,41.130000


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
ASSIGNED_GROUP,,
ITS00329,8234.61000,2360.126923
DC000168,6053.32000,1733.680000
ITS00353,5674.99375,1622.812500
EDC00065,4159.62000,1206.150000
NW000490,3564.91000,1025.910000
...,...,...
EU000203,0.42000,0.420000
NDS00039,0.36000,0.370000
NW000457,0.31000,0.300000


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,,
1083.0,4922.660000,1427.775000
1213.0,794.244130,222.682174
1067.0,757.639091,206.811818
1017.0,672.730000,190.740000
877.0,406.147857,115.354643
1040.0,330.750000,94.657059
992.0,308.160000,86.514706
1123.0,265.573393,73.196786
15833.0,251.928750,67.688125


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,,
1083.0,4922.660000,1427.775000
1213.0,794.244130,222.682174
1067.0,757.639091,206.811818
1017.0,672.730000,190.740000
877.0,406.147857,115.354643
1040.0,330.750000,94.657059
992.0,308.160000,86.514706
1123.0,265.573393,73.196786
15833.0,251.928750,67.688125


In [19]:
"""Group by Priority"""
# Average completion time by status
avgCmplStatus = df_gb_avg(jemInc, ['STATUS', 'PRIORITY'],
                          ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                          ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplStatus.to_csv("./AVGDATA_JEM/jemInc_avgCmplStatus_pr.csv")

# Average completion time by parent service
avgCmplPS = df_gb_avg(jemInc, ['PARENT_SERVICE', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA_JEM/jemInc_avgCmplPS_pr.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(jemInc, ['service', 'PRIORITY'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA_JEM/jemInc_avgCmplServ_pr.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(jemInc, ['ASSIGNED_GROUP', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA_JEM/jemInc_avgCmplAG_pr.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(jemInc, ['org_id', 'PRIORITY'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(jemInc, ['org_id', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA_JEM/jemInc_avgCmplORG_pr.csv")
avgBizORG.to_csv("./AVGDATA_JEM/jemInc_avgBizORG_pr.csv")

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
STATUS   PRIORITY                                                
RESOLVED Low                 3163.196333               905.492667
INPROG   Low                 1911.870000               529.600000
RESOLVED Medium              1518.621765               439.886471
CLOSED   High                 837.543947               837.543947
         Low                  122.606400                34.244421
         Medium                64.994634                17.593648
QUEUED   Low                   28.370000                14.370000
AWAITVEN Medium                 8.396667                 0.000000
         Low                    2.126667                 0.438333
INPROG   Medium                 0.110000                 0.000000
AWAITCUS Low                         NaN                      NaN
QUEUED   Medium                      NaN                      NaN
SLAHOLD  Low                         NaN                      NaN
         Medium                      NaN                      NaN

,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
PARENT_SERVICE,PRIORITY,,
Other Activities,High,10687.821000,10687.821000
Firewall,Low,369.845076,104.116250
Email,Low,289.634545,81.879860
Other Activities,Low,250.155151,77.155713
High-performance Computing,Low,238.201001,68.884260
Mainframe,Low,201.895585,55.723755
Internal Credential Management (ICM),Low,175.780169,47.650000
High-performance Computing,High,147.878621,147.878621
Data Centre Facilities Management,Low,139.458621,38.980805


ACTUAL_COMPLETION_HRS  \
service                           PRIORITY                          
Other - Service Management        High               11245.040965   
Mainframe - zOS & zLinux          Medium              3347.024615   
                                  Low                 1701.756866   
Intra-building Network Services   Low                 1694.976369   
GC WAN National                   Medium              1394.360000   
...                                                           ...   
HPC - Interaction & Visualization Medium                 3.627843   
Intra-building Network Services   High                   3.574118   
Mainframe                         High                   1.740000   
-1                                Medium                 1.185385   
Storage - File                    High                   0.420000   

                                            BUSINESS_COMPLETION_HRS  
service                           PRIORITY                           
Other - Service Management        High                 11245.040965  
Mainframe - zOS & zLinux          Medium                 956.189231  
                                  Low                    485.031343  
Intra-building Network Services   Low                    483.808393  
GC WAN National                   Medium                 403.145000  
...                                                             ...  
HPC - Interaction & Visualization Medium                   0.820196  
Intra-building Network Services   High                     3.574118  
Mainframe                         High                     1.740000  
-1                                Medium                   0.199231  
Storage - File                    High                     0.420000  

[103 rows x 2 columns]

,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
ASSIGNED_GROUP,PRIORITY,,
EDC00031,High,255749.27200,255749.272000
ITS00329,Low,8234.61000,2360.126923
DC000168,Low,6053.32000,1733.680000
ITS00353,Low,5674.99375,1622.812500
NW000490,Low,5567.63000,1589.630000
...,...,...,...
EU000203,High,0.42000,0.420000
NDS00039,Low,0.36000,0.370000
NW000457,Low,0.31000,0.300000


,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,PRIORITY,,
1083.0,Low,4922.660000,1427.775000
1342.0,High,1878.969913,1878.969913
1123.0,Medium,1489.995000,427.850000
1067.0,Low,828.685000,226.576000
1213.0,Low,794.244130,222.682174
...,...,...,...
1047.0,High,10.732409,10.732409
835.0,High,8.170000,8.170000
1057.0,Medium,4.780000,4.770000


,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,PRIORITY,,
1342.0,High,1878.969913,1878.969913
1083.0,Low,4922.660000,1427.775000
1123.0,Medium,1489.995000,427.850000
1058.0,High,307.630000,307.630000
1067.0,Low,828.685000,226.576000
...,...,...,...
17498.0,Low,14.060000,3.520000
1038.0,Low,23.335000,2.815000
1048.0,Low,26.876364,2.704545


In [20]:
"""Group by Priority and Status"""
# Average completion time by parent service
avgCmplPS = df_gb_avg(jemInc, ['PARENT_SERVICE', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA_JEM/jemInc_avgCmplPS_pr_stat.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(jemInc, ['service', 'PRIORITY', 'STATUS'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA_JEM/jemInc_avgCmplServ_pr_stat.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(jemInc, ['ASSIGNED_GROUP', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA_JEM/jemInc_avgCmplAG_pr_stat.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(jemInc, ['org_id', 'PRIORITY', 'STATUS'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(jemInc, ['org_id', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA_JEM/jemInc_avgCmplORG_pr_stat.csv")
avgBizORG.to_csv("./AVGDATA_JEM/jemInc_avgBizORG_pr_stat.csv")

ACTUAL_COMPLETION_HRS  \
PARENT_SERVICE                  PRIORITY STATUS                            
Other Activities                High     CLOSED             10687.821000   
                                Medium   RESOLVED            7805.560000   
High-performance Computing      Low      RESOLVED            3438.489167   
Intra-building Network Services Low      RESOLVED            2425.974000   
High-performance Computing      Low      INPROG              1911.870000   
...                                                                  ...   
Midrange                        Low      SLAHOLD                     NaN   
Mobile Devices                  Medium   SLAHOLD                     NaN   
Other Activities                Low      AWAITCUS                    NaN   
Secure Remote Access            Medium   INPROG                      NaN   
Storage                         Low      QUEUED                      NaN   

                                                   BUSINESS_COMPLETION_HRS  
PARENT_SERVICE                  PRIORITY STATUS                             
Other Activities                High     CLOSED               10687.821000  
                                Medium   RESOLVED              2248.700000  
High-performance Computing      Low      RESOLVED               983.534583  
Intra-building Network Services Low      RESOLVED               695.532000  
High-performance Computing      Low      INPROG                 529.600000  
...                                                                    ...  
Midrange                        Low      SLAHOLD                       NaN  
Mobile Devices                  Medium   SLAHOLD                       NaN  
Other Activities                Low      AWAITCUS                      NaN  
Secure Remote Access            Medium   INPROG                        NaN  
Storage                         Low      QUEUED                        NaN  

[90 rows x 2 columns]

ACTUAL_COMPLETION_HRS  \
service                                            PRIORITY STATUS                            
Other - Service Management                         High     CLOSED             11245.040965   
                                                   Medium   RESOLVED            7805.560000   
IBN - GC LAN                                       Low      RESOLVED            6070.220000   
High-performance Computing                         Low      RESOLVED            3577.381739   
Mainframe - zOS & zLinux                           Medium   CLOSED              3347.024615   
...                                                                                     ...   
Midrange                                           Low      SLAHOLD                     NaN   
Mobile - Enterprise Mobile Device Management (E... Medium   SLAHOLD                     NaN   
Other - Service Management                         Low      AWAITCUS                    NaN   
Secure Remote Access                               Medium   INPROG                      NaN   
Storage                                            Low      QUEUED                      NaN   

                                                                      BUSINESS_COMPLETION_HRS  
service                                            PRIORITY STATUS                             
Other - Service Management                         High     CLOSED               11245.040965  
                                                   Medium   RESOLVED              2248.700000  
IBN - GC LAN                                       Low      RESOLVED              1748.210000  
High-performance Computing                         Low      RESOLVED              1022.646522  
Mainframe - zOS & zLinux                           Medium   CLOSED                 956.189231  
...                                                                                       ...  
Midrange                                           Low      SLAHOLD                       NaN  
Mobile - Enterprise Mobile Device Management (E... Medium   SLAHOLD                       NaN  
Other - Service Management                         Low      AWAITCUS                      NaN  
Secure Remote Access                               Medium   INPROG                        NaN  
Storage                                            Low      QUEUED                        NaN  

[146 rows x 2 columns]

ACTUAL_COMPLETION_HRS  \
ASSIGNED_GROUP PRIORITY STATUS                            
EDC00031       High     CLOSED               255749.272   
ITS00329       Low      CLOSED                 8234.610   
SM000537       Medium   RESOLVED               7805.560   
NW000452       Low      RESOLVED               6070.220   
DC000168       Low      CLOSED                 6053.320   
...                                                 ...   
NW000453       Low      QUEUED                      NaN   
NW000481       Low      INPROG                      NaN   
                        QUEUED                      NaN   
NW000490       Low      INPROG                      NaN   
                        QUEUED                      NaN   

                                  BUSINESS_COMPLETION_HRS  
ASSIGNED_GROUP PRIORITY STATUS                             
EDC00031       High     CLOSED              255749.272000  
ITS00329       Low      CLOSED                2360.126923  
SM000537       Medium   RESOLVED              2248.700000  
NW000452       Low      RESOLVED              1748.210000  
DC000168       Low      CLOSED                1733.680000  
...                                                   ...  
NW000453       Low      QUEUED                        NaN  
NW000481       Low      INPROG                        NaN  
                        QUEUED                        NaN  
NW000490       Low      INPROG                        NaN  
                        QUEUED                        NaN  

[453 rows x 2 columns]

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
org_id PRIORITY STATUS                                                  
1055.0 Medium   RESOLVED            7805.560000              2248.700000
1083.0 Low      CLOSED              4922.660000              1427.775000
1062.0 Low      RESOLVED            3438.489167               983.534583
1342.0 Medium   RESOLVED            2274.060000               654.050000
       Low      RESOLVED            2062.025000               593.325000
...                                         ...                      ...
                AWAITVEN                    NaN                      NaN
                INPROG                      NaN                      NaN
                SLAHOLD                     NaN                      NaN
       Medium   AWAITVEN                    NaN                      NaN
1355.0 Low      AWAITCUS                    NaN                      NaN

[109 rows x 2 columns]

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
org_id PRIORITY STATUS                                                  
1055.0 Medium   RESOLVED            7805.560000              2248.700000
1342.0 High     CLOSED              1878.969913              1878.969913
1083.0 Low      CLOSED              4922.660000              1427.775000
1062.0 Low      RESOLVED            3438.489167               983.534583
1342.0 Medium   RESOLVED            2274.060000               654.050000
...                                         ...                      ...
       Low      AWAITVEN                    NaN                      NaN
                INPROG                      NaN                      NaN
                SLAHOLD                     NaN                      NaN
       Medium   AWAITVEN                    NaN                      NaN
1355.0 Low      AWAITCUS                    NaN                      NaN

[109 rows x 2 columns]

In [21]:
"""Excluding Event Management"""
# Filter for event management tickets
eemInc = incidents[incidents["EXTERNAL_SYSTEM"] != "EVENTMANAGEMENT"]

# Reset index
eemInc = jemInc.reset_index(drop=True)

# Filter Incident owner history for event management tickets
eemIOH = incOwnHist[incOwnHist["ticket_nmbr"].isin(jemInc["TICKET_NMBR"])]

# Reset index
eemIOH = jemIOH.reset_index(drop=True)

# Display
display(eemInc)
display(eemIOH)

,TICKET_NMBR,PARENT_SERVICE,service,org_id,ASSIGNED_GROUP,OPEN_DATE,CLOSE_DATE,PRIORITY,STATUS,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS,AGING,CLASS_STRUCTURE_ID,class_structure,CLASSIFICATION_ID,classification,EXTERNAL_SYSTEM,GLOBAL_TICKET_ID,CLOSURE_CODE,LAST_MODIFIED_DATE
0,IN10028612,Middleware,Middleware,1342.0,DC000152,01APR2016:00:01:16,01APR2016:08:26:30,Low,CLOSED,8.42,1.43,NaN,2101,IT Issue \ Hardware,2101,Hardware Issue,EVENTMANAGEMENT,NaN,OTH,12APR2016:05:30:11
1,IN10028613,Midrange,Midrange - Windows -X86,1342.0,NDS00032,01APR2016:00:35:21,11APR2016:11:06:12,Low,CLOSED,250.51,64.10,NaN,21,IT Issue,21,IT Issue,EVENTMANAGEMENT,NaN,CIS,22APR2016:05:30:33
2,IN10028615,High-performance Computing,High-performance Computing,1342.0,DC000133,01APR2016:02:57:00,01APR2016:07:42:08,Low,CLOSED,4.75,0.70,NaN,3102,IT Request \ Software Request,3102,Software Request,EVENTMANAGEMENT,NaN,OSD,12APR2016:05:30:10
3,IN10028617,Midrange,Midrange - Windows -X86,1342.0,DC000115,01APR2016:06:21:10,18APR2016:13:09:13,Medium,CLOSED,414.80,116.15,NaN,2101,IT Issue \ Hardware,2101,Hardware Issue,EVENTMANAGEMENT,NaN,BSR,29APR2016:05:30:36
4,IN10028618,Other Activities,Other - Service Management,1342.0,ESI00011,01APR2016:06:30:34,17APR2016:19:40:25,Medium,CLOSED,397.16,110.00,NaN,2102,IT Issue \ Software,2102,Software Issue,EVENTMANAGEMENT,NaN,INF,28APR2016:05:30:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96960,IN10293537,Intra-building Network Services,IBN - GC LAN,1047.0,NW000405,30NOV2021:20:54:25,10DEC2021:08:38:54,Medium,RESOLVED,227.74,71.63,NaN,2103,IT Issue \ Networks,2103,Networks Issue,EVENTMANAGEMENT,NaN,RWOI,10DEC2021:08:38:54
96961,IN10293538,Middleware,Middleware,1047.0,DC000152,30NOV2021:21:21:23,03DEC2021:06:42:48,Low,CLOSED,57.36,20.00,NaN,2102,IT Issue \ Software,2102,Software Issue,EVENTMANAGEMENT,NaN,RP,14DEC2021:05:30:16
96962,IN10293540,High-performance Computing,High-performance Computing,1062.0,DC000132,30NOV2021:21:42:14,NaN,Low,AWAITVEN,NaN,NaN,19.658366,2102,IT Issue \ Software,2102,Software Issue,EVENTMANAGEMENT,NaN,RP,30NOV2021:21:51:26
96963,IN10293541,Midrange,Midrange,1342.0,DC000209,30NOV2021:21:44:01,01DEC2021:08:37:06,Low,CLOSED,10.88,1.62,NaN,21,IT Issue,21,IT Issue,EVENTMANAGEMENT,NaN,RP,12DEC2021:05:30:09


,ticket_nmbr,STATUS,assigned_group,PARENT_SERVICE,service,CHANGE_DATE,TIME_IN_STATUS_BY_OWNER_HRS
0,IN10028612,QUEUED,DC000152,Middleware,Middleware,01APR2016:00:03:12,8.383333
1,IN10028612,RESOLVED,DC000152,Middleware,Middleware,01APR2016:08:26:30,0.000000
2,IN10028612,CLOSED,DC000152,Middleware,Middleware,12APR2016:05:30:11,NaN
3,IN10028613,QUEUED,NDS00032,Midrange,Midrange - Windows -X86,01APR2016:01:04:45,64.100000
4,IN10028613,RESOLVED,NDS00032,Midrange,Midrange - Windows -X86,11APR2016:11:06:12,0.000000
...,...,...,...,...,...,...,...
420311,IN10293541,NEW,NaN,NaN,NaN,NaN,NaN
420312,IN10293552,INPROG,SM000562,Other Activities,Other - Service Management,NaN,0.000000
420313,IN10293552,QUEUED,SM000562,Other Activities,Other - Service Management,NaN,NaN
420314,IN10293552,INPROG,DC000155,Mainframe,Mainframe,NaN,NaN


In [22]:
# Average time incidents spend in particular status
avgTimeStatus = df_gb_avg(eemIOH, ['STATUS'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeStatus.to_csv("./AVGDATA_EEM/eemIOH_avgTimeStatus.csv")

# Average time incidents spend with assigned group
avgTimeAG = df_gb_avg(eemIOH, ['assigned_group'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA_EEM/eemIOH_avgTimeAG.csv")

# Average time incidents spend by parent service
avgTimePS = df_gb_avg(eemIOH, ['PARENT_SERVICE'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimePS.to_csv("./AVGDATA_EEM/eemIOH_avgTimePS.csv")

# Average time incidents spend by service
avgTimeAG = df_gb_avg(eemIOH, ['service'], ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA_EEM/eemIOH_avgTimeServ.csv")

,TIME_IN_STATUS_BY_OWNER_HRS
STATUS,
AWAITVEN,167.117355
PENDINGVEN,88.863389
SLAHOLD,73.841899
AWAITCHG,66.540715
AWAITCUS,58.563833
PENDINGREV,48.973810
PENDINGCHG,40.213333
PENDING,32.912584
PENDINGCUS,23.423246


,TIME_IN_STATUS_BY_OWNER_HRS
assigned_group,
ESI00052,2772.372917
ITS00352,2502.758333
DC000145,1724.883333
NW000469,1639.416667
ITS00329,1422.923016
...,...
EDC00041,0.000000
NW000467,0.000000
DC000226,NaN


,TIME_IN_STATUS_BY_OWNER_HRS
PARENT_SERVICE,
Bulk Print,1724.883333
Distributed Print,319.450000
Classified Infrastructure,290.000000
Firewall,44.564273
Managed Secure File Transfer,26.101613
High-performance Computing,25.617979
Email,24.196342
Database,21.292119
Internal Credential Management (ICM),18.915646


,TIME_IN_STATUS_BY_OWNER_HRS
service,
Bulk Print,1724.883333
Mobile Devices,638.362500
Distributed Print,319.450000
Classified Infrastructure - GCSI - Hosting Environment,290.000000
Mainframe - zOS & zLinux,251.863226
Intra-building Network Services,122.159201
Other - Security,93.827952
Firewall,44.564273
Other - Internal Services & Strategy,42.347312


In [23]:
"""Group By Status"""
# Average time incidents spend with assigned group
avgTimeAG = df_gb_avg(eemIOH, ['assigned_group', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA_EEM/eemIOH_avgTimeAGStatus.csv")

# Average time incidents spend by parent service
avgTimePS = df_gb_avg(eemIOH, ['PARENT_SERVICE', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimePS.to_csv("./AVGDATA_EEM/eemIOH_avgTimePSStatus.csv")

# Average time incidents spend by service
avgTimeAG = df_gb_avg(eemIOH, ['service', 'STATUS'],
                      ['TIME_IN_STATUS_BY_OWNER_HRS'], ['TIME_IN_STATUS_BY_OWNER_HRS'])

# Save to file
avgTimeAG.to_csv("./AVGDATA_EEM/eemIOH_avgTimeServStatus.csv")

TIME_IN_STATUS_BY_OWNER_HRS
assigned_group STATUS                               
DC000118       QUEUED                    5559.466667
ITS00340       AWAITCUS                  5552.816667
ESI00052       AWAITCUS                  5550.833333
ITS00329       AWAITCUS                  5446.583333
               QUEUED                    3490.685714
...                                              ...
SM000562       CLOSED                            NaN
SM000585       AWAITCUS                          NaN
               CLOSED                            NaN
SM000586       CLOSED                            NaN
TIVMON         CLOSED                            NaN

[1480 rows x 1 columns]

TIME_IN_STATUS_BY_OWNER_HRS
PARENT_SERVICE                    STATUS                                 
Bulk Print                        QUEUED                      2111.008333
                                  INPROG                       952.633333
Secure Remote Access              AWAITCUS                     727.808333
Data Centre Facilities Management PENDINGVEN                   584.444444
Email                             AWAITCUS                     570.076667
...                                                                   ...
Other Activities                  CLOSED                              NaN
Secure Remote Access              AWAITCHG                            NaN
                                  CLOSED                              NaN
Storage                           CLOSED                              NaN
Workplace Technology Services     CLOSED                              NaN

[221 rows x 1 columns]

TIME_IN_STATUS_BY_OWNER_HRS
service                       STATUS                               
Bulk Print                    QUEUED                    2111.008333
Other - Security              AWAITCUS                  1468.991667
Mobile Devices                INPROG                    1321.333333
                              QUEUED                    1010.000000
Bulk Print                    INPROG                     952.633333
...                                                             ...
Secure Remote Access          AWAITCHG                          NaN
                              CLOSED                            NaN
Storage                       CLOSED                            NaN
Storage - File                CLOSED                            NaN
Workplace Technology Services CLOSED                            NaN

[365 rows x 1 columns]

In [24]:
# Average completion time by status
avgCmplStatus = df_gb_avg(eemInc, ['STATUS'],
                          ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                          ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplStatus.to_csv("./AVGDATA_EEM/eemInc_avgCmplStatus.csv")

# Average completion time by parent service
avgCmplPS = df_gb_avg(eemInc, ['PARENT_SERVICE'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA_EEM/eemInc_avgCmplPS.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(eemInc, ['service'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA_EEM/eemInc_avgCmplServ.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(eemInc, ['ASSIGNED_GROUP'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA_EEM/eemInc_avgCmplAG.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(eemInc, ['org_id'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(eemInc, ['org_id'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA_EEM/eemInc_avgCmplORG.csv")
avgBizORG.to_csv("./AVGDATA_EEM/eemInc_avgBizORG.csv")

,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
STATUS,,
RESOLVED,2568.350213,737.081915
INPROG,955.990000,264.800000
CLOSED,122.551871,43.918936
QUEUED,28.370000,14.370000
AWAITVEN,4.216667,0.292222
AWAITCUS,NaN,NaN
SLAHOLD,NaN,NaN


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
PARENT_SERVICE,,
Firewall,334.168306,94.297132
Other Activities,311.433446,187.417807
High-performance Computing,228.265096,66.235825
Email,197.513241,55.943657
Internal Credential Management (ICM),173.175082,46.918033
Intra-building Network Services,134.191680,36.432287
Database,113.763271,32.698271
Workplace Technology Services,108.331128,27.255668
Mainframe,101.080406,27.666743


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
service,,
Mainframe - zOS & zLinux,1969.112875,561.594500
Intra-building Network Services,1303.024045,372.335909
Other - Security,494.226139,140.868399
Firewall,334.168306,94.297132
Other - Service Management,320.901298,206.286496
High-performance Computing,314.712877,90.003782
Email,197.513241,55.943657
Internal Credential Management (ICM),173.175082,46.918033
Other - Project Management & Delivery,153.010000,41.130000


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
ASSIGNED_GROUP,,
ITS00329,8234.61000,2360.126923
DC000168,6053.32000,1733.680000
ITS00353,5674.99375,1622.812500
EDC00065,4159.62000,1206.150000
NW000490,3564.91000,1025.910000
...,...,...
EU000203,0.42000,0.420000
NDS00039,0.36000,0.370000
NW000457,0.31000,0.300000


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,,
1083.0,4922.660000,1427.775000
1213.0,794.244130,222.682174
1067.0,757.639091,206.811818
1017.0,672.730000,190.740000
877.0,406.147857,115.354643
1040.0,330.750000,94.657059
992.0,308.160000,86.514706
1123.0,265.573393,73.196786
15833.0,251.928750,67.688125


,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,,
1083.0,4922.660000,1427.775000
1213.0,794.244130,222.682174
1067.0,757.639091,206.811818
1017.0,672.730000,190.740000
877.0,406.147857,115.354643
1040.0,330.750000,94.657059
992.0,308.160000,86.514706
1123.0,265.573393,73.196786
15833.0,251.928750,67.688125


In [25]:
"""Group by Priority"""
# Average completion time by status
avgCmplStatus = df_gb_avg(eemInc, ['STATUS', 'PRIORITY'],
                          ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                          ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplStatus.to_csv("./AVGDATA_EEM/eemInc_avgCmplStatus_pr.csv")

# Average completion time by parent service
avgCmplPS = df_gb_avg(eemInc, ['PARENT_SERVICE', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA_EEM/eemInc_avgCmplPS_pr.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(eemInc, ['service', 'PRIORITY'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA_EEM/eemInc_avgCmplServ_pr.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(eemInc, ['ASSIGNED_GROUP', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA_EEM/eemInc_avgCmplAG_pr.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(eemInc, ['org_id', 'PRIORITY'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(eemInc, ['org_id', 'PRIORITY'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA_EEM/eemInc_avgCmplORG_pr.csv")
avgBizORG.to_csv("./AVGDATA_EEM/eemInc_avgBizORG_pr.csv")

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
STATUS   PRIORITY                                                
RESOLVED Low                 3163.196333               905.492667
INPROG   Low                 1911.870000               529.600000
RESOLVED Medium              1518.621765               439.886471
CLOSED   High                 837.543947               837.543947
         Low                  122.606400                34.244421
         Medium                64.994634                17.593648
QUEUED   Low                   28.370000                14.370000
AWAITVEN Medium                 8.396667                 0.000000
         Low                    2.126667                 0.438333
INPROG   Medium                 0.110000                 0.000000
AWAITCUS Low                         NaN                      NaN
QUEUED   Medium                      NaN                      NaN
SLAHOLD  Low                         NaN                      NaN
         Medium                      NaN                      NaN

,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
PARENT_SERVICE,PRIORITY,,
Other Activities,High,10687.821000,10687.821000
Firewall,Low,369.845076,104.116250
Email,Low,289.634545,81.879860
Other Activities,Low,250.155151,77.155713
High-performance Computing,Low,238.201001,68.884260
Mainframe,Low,201.895585,55.723755
Internal Credential Management (ICM),Low,175.780169,47.650000
High-performance Computing,High,147.878621,147.878621
Data Centre Facilities Management,Low,139.458621,38.980805


ACTUAL_COMPLETION_HRS  \
service                           PRIORITY                          
Other - Service Management        High               11245.040965   
Mainframe - zOS & zLinux          Medium              3347.024615   
                                  Low                 1701.756866   
Intra-building Network Services   Low                 1694.976369   
GC WAN National                   Medium              1394.360000   
...                                                           ...   
HPC - Interaction & Visualization Medium                 3.627843   
Intra-building Network Services   High                   3.574118   
Mainframe                         High                   1.740000   
-1                                Medium                 1.185385   
Storage - File                    High                   0.420000   

                                            BUSINESS_COMPLETION_HRS  
service                           PRIORITY                           
Other - Service Management        High                 11245.040965  
Mainframe - zOS & zLinux          Medium                 956.189231  
                                  Low                    485.031343  
Intra-building Network Services   Low                    483.808393  
GC WAN National                   Medium                 403.145000  
...                                                             ...  
HPC - Interaction & Visualization Medium                   0.820196  
Intra-building Network Services   High                     3.574118  
Mainframe                         High                     1.740000  
-1                                Medium                   0.199231  
Storage - File                    High                     0.420000  

[103 rows x 2 columns]

,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
ASSIGNED_GROUP,PRIORITY,,
EDC00031,High,255749.27200,255749.272000
ITS00329,Low,8234.61000,2360.126923
DC000168,Low,6053.32000,1733.680000
ITS00353,Low,5674.99375,1622.812500
NW000490,Low,5567.63000,1589.630000
...,...,...,...
EU000203,High,0.42000,0.420000
NDS00039,Low,0.36000,0.370000
NW000457,Low,0.31000,0.300000


,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,PRIORITY,,
1083.0,Low,4922.660000,1427.775000
1342.0,High,1878.969913,1878.969913
1123.0,Medium,1489.995000,427.850000
1067.0,Low,828.685000,226.576000
1213.0,Low,794.244130,222.682174
...,...,...,...
1047.0,High,10.732409,10.732409
835.0,High,8.170000,8.170000
1057.0,Medium,4.780000,4.770000


,,ACTUAL_COMPLETION_HRS,BUSINESS_COMPLETION_HRS
org_id,PRIORITY,,
1342.0,High,1878.969913,1878.969913
1083.0,Low,4922.660000,1427.775000
1123.0,Medium,1489.995000,427.850000
1058.0,High,307.630000,307.630000
1067.0,Low,828.685000,226.576000
...,...,...,...
17498.0,Low,14.060000,3.520000
1038.0,Low,23.335000,2.815000
1048.0,Low,26.876364,2.704545


In [26]:
"""Group by Priority and Status"""
# Average completion time by parent service
avgCmplPS = df_gb_avg(eemInc, ['PARENT_SERVICE', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplPS.to_csv("./AVGDATA_EEM/eemInc_avgCmplPS_pr_stat.csv")

# Average completion time by service
avgCmplServ = df_gb_avg(eemInc, ['service', 'PRIORITY', 'STATUS'],
                        ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                        ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplServ.to_csv("./AVGDATA_EEM/eemInc_avgCmplServ_pr_stat.csv")

# Average completion time by assigned group
avgCmplAG = df_gb_avg(eemInc, ['ASSIGNED_GROUP', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['ACTUAL_COMPLETION_HRS'])

# Save to file
avgCmplAG.to_csv("./AVGDATA_EEM/eemInc_avgCmplAG_pr_stat.csv")

# Average completion time by org id
# Note that sorting by business hours is different, probably due to data input error
avgCmplORG = df_gb_avg(eemInc, ['org_id', 'PRIORITY', 'STATUS'],
                       ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                       ['ACTUAL_COMPLETION_HRS'])

avgBizORG = df_gb_avg(eemInc, ['org_id', 'PRIORITY', 'STATUS'],
                      ['ACTUAL_COMPLETION_HRS', 'BUSINESS_COMPLETION_HRS'],
                      ['BUSINESS_COMPLETION_HRS'])

# Save to file
avgCmplORG.to_csv("./AVGDATA_EEM/eemInc_avgCmplORG_pr_stat.csv")
avgBizORG.to_csv("./AVGDATA_EEM/eemInc_avgBizORG_pr_stat.csv")

ACTUAL_COMPLETION_HRS  \
PARENT_SERVICE                  PRIORITY STATUS                            
Other Activities                High     CLOSED             10687.821000   
                                Medium   RESOLVED            7805.560000   
High-performance Computing      Low      RESOLVED            3438.489167   
Intra-building Network Services Low      RESOLVED            2425.974000   
High-performance Computing      Low      INPROG              1911.870000   
...                                                                  ...   
Midrange                        Low      SLAHOLD                     NaN   
Mobile Devices                  Medium   SLAHOLD                     NaN   
Other Activities                Low      AWAITCUS                    NaN   
Secure Remote Access            Medium   INPROG                      NaN   
Storage                         Low      QUEUED                      NaN   

                                                   BUSINESS_COMPLETION_HRS  
PARENT_SERVICE                  PRIORITY STATUS                             
Other Activities                High     CLOSED               10687.821000  
                                Medium   RESOLVED              2248.700000  
High-performance Computing      Low      RESOLVED               983.534583  
Intra-building Network Services Low      RESOLVED               695.532000  
High-performance Computing      Low      INPROG                 529.600000  
...                                                                    ...  
Midrange                        Low      SLAHOLD                       NaN  
Mobile Devices                  Medium   SLAHOLD                       NaN  
Other Activities                Low      AWAITCUS                      NaN  
Secure Remote Access            Medium   INPROG                        NaN  
Storage                         Low      QUEUED                        NaN  

[90 rows x 2 columns]

ACTUAL_COMPLETION_HRS  \
service                                            PRIORITY STATUS                            
Other - Service Management                         High     CLOSED             11245.040965   
                                                   Medium   RESOLVED            7805.560000   
IBN - GC LAN                                       Low      RESOLVED            6070.220000   
High-performance Computing                         Low      RESOLVED            3577.381739   
Mainframe - zOS & zLinux                           Medium   CLOSED              3347.024615   
...                                                                                     ...   
Midrange                                           Low      SLAHOLD                     NaN   
Mobile - Enterprise Mobile Device Management (E... Medium   SLAHOLD                     NaN   
Other - Service Management                         Low      AWAITCUS                    NaN   
Secure Remote Access                               Medium   INPROG                      NaN   
Storage                                            Low      QUEUED                      NaN   

                                                                      BUSINESS_COMPLETION_HRS  
service                                            PRIORITY STATUS                             
Other - Service Management                         High     CLOSED               11245.040965  
                                                   Medium   RESOLVED              2248.700000  
IBN - GC LAN                                       Low      RESOLVED              1748.210000  
High-performance Computing                         Low      RESOLVED              1022.646522  
Mainframe - zOS & zLinux                           Medium   CLOSED                 956.189231  
...                                                                                       ...  
Midrange                                           Low      SLAHOLD                       NaN  
Mobile - Enterprise Mobile Device Management (E... Medium   SLAHOLD                       NaN  
Other - Service Management                         Low      AWAITCUS                      NaN  
Secure Remote Access                               Medium   INPROG                        NaN  
Storage                                            Low      QUEUED                        NaN  

[146 rows x 2 columns]

ACTUAL_COMPLETION_HRS  \
ASSIGNED_GROUP PRIORITY STATUS                            
EDC00031       High     CLOSED               255749.272   
ITS00329       Low      CLOSED                 8234.610   
SM000537       Medium   RESOLVED               7805.560   
NW000452       Low      RESOLVED               6070.220   
DC000168       Low      CLOSED                 6053.320   
...                                                 ...   
NW000453       Low      QUEUED                      NaN   
NW000481       Low      INPROG                      NaN   
                        QUEUED                      NaN   
NW000490       Low      INPROG                      NaN   
                        QUEUED                      NaN   

                                  BUSINESS_COMPLETION_HRS  
ASSIGNED_GROUP PRIORITY STATUS                             
EDC00031       High     CLOSED              255749.272000  
ITS00329       Low      CLOSED                2360.126923  
SM000537       Medium   RESOLVED              2248.700000  
NW000452       Low      RESOLVED              1748.210000  
DC000168       Low      CLOSED                1733.680000  
...                                                   ...  
NW000453       Low      QUEUED                        NaN  
NW000481       Low      INPROG                        NaN  
                        QUEUED                        NaN  
NW000490       Low      INPROG                        NaN  
                        QUEUED                        NaN  

[453 rows x 2 columns]

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
org_id PRIORITY STATUS                                                  
1055.0 Medium   RESOLVED            7805.560000              2248.700000
1083.0 Low      CLOSED              4922.660000              1427.775000
1062.0 Low      RESOLVED            3438.489167               983.534583
1342.0 Medium   RESOLVED            2274.060000               654.050000
       Low      RESOLVED            2062.025000               593.325000
...                                         ...                      ...
                AWAITVEN                    NaN                      NaN
                INPROG                      NaN                      NaN
                SLAHOLD                     NaN                      NaN
       Medium   AWAITVEN                    NaN                      NaN
1355.0 Low      AWAITCUS                    NaN                      NaN

[109 rows x 2 columns]

ACTUAL_COMPLETION_HRS  BUSINESS_COMPLETION_HRS
org_id PRIORITY STATUS                                                  
1055.0 Medium   RESOLVED            7805.560000              2248.700000
1342.0 High     CLOSED              1878.969913              1878.969913
1083.0 Low      CLOSED              4922.660000              1427.775000
1062.0 Low      RESOLVED            3438.489167               983.534583
1342.0 Medium   RESOLVED            2274.060000               654.050000
...                                         ...                      ...
       Low      AWAITVEN                    NaN                      NaN
                INPROG                      NaN                      NaN
                SLAHOLD                     NaN                      NaN
       Medium   AWAITVEN                    NaN                      NaN
1355.0 Low      AWAITCUS                    NaN                      NaN

[109 rows x 2 columns]